In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plottable import Table

from reportlab.platypus import SimpleDocTemplate, Table as RLTable, TableStyle
from reportlab.lib import colors

### STEP 1: Load in  CSV file 

In [ ]:
#replace this with the path to the file
file_path = "/Users/iz132/Desktop/delibs.csv"
df = pd.read_csv(file_path)

In [72]:
df = df.rename(columns={
    "Timestamp": "timestamp",
    "Board member": "board_member",
    "Auditionee Name": "auditionee",
    "Rating [Accuracy]": "accuracy",
    "Rating [Energy ]": "energy",
    "Rating [Performance ]": "performance",
    "Rating [Improvement ]": "improvement",
    "Initial Vote ": "initial_vote",
    "Additional Comments?": "comments"
})


### Step 2: Score Calculation 

In [ ]:
# Calculate percentage of initial yes votes
df["initial_vote_num"] = (
    df["initial_vote"].astype(str).str.strip().str.lower().map({"yes": 1, "no": 0})
)

# Calculate composite scores (accuracy + energy + performance)
avg_composite = (
    df.groupby("auditionee")
    .agg(
        accuracy_score=("accuracy", "mean"),
        energy_score=("energy", "mean"),
        performance_score=("performance", "mean"),
        pct_yes=("initial_vote_num", lambda s: round(s.mean() * 100, 2)),
        yes_count=("initial_vote_num", "sum"),
    )
    .reset_index()
)

avg_composite["composite_score"] = (
    avg_composite["accuracy_score"]
    + avg_composite["energy_score"]
    + avg_composite["performance_score"]
)

# Calculate improvement scores
avg_improvement = (
    df.groupby("auditionee")
    .agg(
        improvement_avg=("improvement", lambda s: round(s.mean(), 2)),
        pct_yes=("initial_vote_num", lambda s: round(s.mean() * 100, 2)),
    )
    .reset_index()
)

### Step 3: Display Results

In [ ]:
# Display all auditionees sorted by composite score
n = df["auditionee"].nunique()
display(
    avg_composite[["auditionee", "composite_score", "pct_yes"]]
    .sort_values(by="composite_score", ascending=False)
    .head(n)
    .reset_index(drop=True)
)

In [ ]:
# Display auditionees sorted by improvement score
display(
    avg_improvement[["auditionee", "improvement_avg", "pct_yes"]]
    .sort_values(by="improvement_avg", ascending=False)
    .reset_index(drop=True)
)

In [ ]:
# Find auditionees in top N of both composite and improvement scores
N = 20

topN_composite = (
    avg_composite.sort_values(by="composite_score", ascending=False)
    .head(N)["auditionee"]
)

topN_improvement = (
    avg_improvement.sort_values(by="improvement_avg", ascending=False)
    .head(N)["auditionee"]
)

# Intersection (who’s in both)
common_topN = set(topN_composite).intersection(set(topN_improvement))

print(f"Auditionees in top {N} of BOTH lists ({len(common_topN)} total):")
for name in sorted(common_topN):
    print("-", name)